In [1]:
import logging
import sys

import pandas as pd
import numpy as np
import yaml

sys.path.append('/home/yuliya/repos/cosybio/FedProt/FedProt')
from client import Client
import utils

logging.basicConfig(
    level=logging.DEBUG, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", datefmt="%d-%b-%y %H:%M:%S"
)

24-May-24 18:40:59 - rpy2.situation - INFO - cffi mode is CFFI_MODE.ANY
24-May-24 18:40:59 - rpy2.situation - DEBUG - Looking for R home with: R RHOME
24-May-24 18:40:59 - rpy2.situation - INFO - R home found: /home/yuliya/anaconda3/envs/fedprot/lib/R
24-May-24 18:40:59 - rpy2.situation - DEBUG - Looking for LD_LIBRARY_PATH with: /home/yuliya/anaconda3/envs/fedprot/lib/R/bin/Rscript -e cat(Sys.getenv("LD_LIBRARY_PATH"))
24-May-24 18:41:00 - rpy2.situation - INFO - R library path: 
24-May-24 18:41:00 - rpy2.situation - INFO - LD_LIBRARY_PATH: 
24-May-24 18:41:00 - rpy2.rinterface_lib.ffi_proxy - DEBUG - cffi mode is InterfaceType.API
24-May-24 18:41:00 - rpy2.rinterface_lib.embedded - INFO - Default options to initialize R: rpy2, --quiet, --no-save
24-Mai-24 18:41:00 - rpy2.rinterface_lib.embedded - INFO - R is already initialized. No need to initialize.


# Base Config

In [2]:
data_dir = "/home/yuliya/repos/cosybio/FedProt/data/TMT_data/"
MODE = "01_smaller_lib_balanced_PG_MajorPG"
cohorts = ["Center1", "Center2", "Center3"]
output_path = "/home/yuliya/repos/cosybio/FedProt/evaluation/TMT_data/"


data_dir = f"{data_dir}/{MODE}"  # path to data folder
output_path = f"{output_path}/{MODE}/results"  # path to output folder

In [3]:
# data_dir = "/home/yuliya/repos/cosybio/random_tests_scripts/2024_05_Added_zero_features_FedProt/"
# MODE = "data"
# cohorts = ["Center1", "Center2"]#, "Center3"]
# output_path = "/home/yuliya/repos/cosybio/random_tests_scripts/2024_05_Added_zero_features_FedProt/"


# data_dir = f"{data_dir}/{MODE}"  # path to data folder
# output_path = f"{output_path}/{MODE}/results"  # path to output folder

In [4]:
with open(f"{data_dir}/{cohorts[0]}/config.yml", 'r') as file:
    config = yaml.safe_load(file)

# parse config
intensity_path = config["fedprot"]["intensities"]
design_path = config["fedprot"]["design"]
use_counts = config["fedprot"]["use_counts"]

if use_counts:
    counts_path = config["fedprot"]["counts"]
else:
    counts_path = None
result_name = config["fedprot"]["result_table"]

max_na_rate = config["fedprot"]["max_na_rate"]
log_transformed = config["fedprot"]["log_transformed"]

experiment_type = config["fedprot"]["experiment_type"]

if experiment_type == "TMT":
    ref_type = config["fedprot"]["ref_type"]
    plex_covariate = config["fedprot"]["plex_covariate"]
    plex_column = config["fedprot"]["plex_column"]

    use_median = config["fedprot"]["use_median"]
    use_irs = config["fedprot"]["use_irs"]

remove_single_pep_protein = config["fedprot"]["remove_single_pep_protein"]
target_classes = config["fedprot"]["target_classes"]
covariates = config["fedprot"]["covariates"]

only_shared_proteins = config["fedprot"]["only_shared_proteins"]

print("Loading data")
# print config  
print(max_na_rate, log_transformed, experiment_type)
print(plex_covariate, use_median, use_irs, remove_single_pep_protein)
print(target_classes, covariates)

Loading data
0.8 False TMT
True True True True
['heathy', 'FSGS'] []


# Initialize clients

In [5]:
store_clients = {}

prot_names = []
if experiment_type == "TMT":
    if plex_covariate:
        plex_covariates_list = []

if use_counts:
    list_of_counts = []

# clinets are joining
for cohort_name in cohorts:
    intensity_file_path = f"{data_dir}/{cohort_name}/{intensity_path}"
    design_file_path = f"{data_dir}/{cohort_name}/{design_path}"

    if use_counts:
        count_file_path = f"{data_dir}/{cohort_name}/{counts_path}"
    else:
        count_file_path = None
    
    # Initialize the client
    client = Client(
        cohort_name,
        intensity_file_path,
        count_file_path,
        design_file_path,
        experiment_type,
        log_transformed = log_transformed,
        ref_type = ref_type,
        plex_column = plex_column,
        target_classes = target_classes,
    )
    store_clients[client.cohort_name] = client

    if experiment_type == "TMT":
        if plex_covariate:
            plex_covariates_list += client.tmt_names

    if use_counts:
        min_counts = client.get_min_count()
        list_of_counts.append(min_counts.to_dict())
        logging.info("Min counts have been computed...")

    # add list (as list of lists) of protein names
    prot_names.append(client.prot_names)

logging.info(f"Plexes: {plex_covariates_list}")

24-Mai-24 18:41:00 - root - INFO - Client Center1: Using counts.
24-Mai-24 18:41:00 - root - INFO - Client Center1: Using TMT-plex column 'Pool'.
24-Mai-24 18:41:00 - root - INFO - Client Center1: Checking TMT-plex Pool1 for single non-NA values.
24-Mai-24 18:41:00 - root - INFO - Client Center1: Checking TMT-plex Pool2 for single non-NA values.
24-Mai-24 18:41:00 - root - INFO - Client Center1: Found 2 TMT-plexes. Plexes: ['Pool1', 'Pool2']
24-Mai-24 18:41:00 - root - INFO - Client Center1: TMT data loaded successfully.
24-Mai-24 18:41:00 - root - INFO - Client Center1: Replacing rows with only one non-NA value with NA.
24-Mai-24 18:41:00 - root - INFO - Client Center1: Rows will be affected : 0
24-Mai-24 18:41:00 - root - INFO - Client Center1: TMT data will be log2 transformed after normalization.
24-Mai-24 18:41:00 - root - INFO - Client Center1: Samples are filtered based on target classes.
24-Mai-24 18:41:00 - root - INFO - Client Center1: 20 samples are kept.
24-Mai-24 18:41:00 

In [6]:
# SERVER SIDE
prot_names = utils.get_analyzed_proteins(prot_names, only_shared_proteins)
logging.info(f"SERVER: Common proteins: {len(prot_names)}")

variables = target_classes + covariates

if use_counts:
    min_counts = list()
    for local_counts in list_of_counts:
        min_counts.append(pd.DataFrame.from_dict(local_counts, orient='index'))
    global_min_counts = pd.concat(min_counts, axis=1)
    global_min_counts = global_min_counts.min(axis=1)
    global_min_counts = global_min_counts.reindex(prot_names, fill_value=0)
    logging.info("Min counts have been aggregated..., shape: " + str(global_min_counts.shape))

24-Mai-24 18:41:00 - root - INFO - SERVER: Common proteins: 621
24-Mai-24 18:41:00 - root - INFO - Min counts have been aggregated..., shape: (621,)


# Validate inputs

In [7]:
# CLIENT SIDE
# validate inputs and add cohort effects
# apply filters

list_of_na_counts_tuples = []
total_samples = {}

for c in cohorts:
    client = store_clients[c]

    if use_counts:
        client.counts = global_min_counts.copy()
    client.validate_inputs(prot_names, variables)

    # add cohort effect columns to each design matrix
    # if plex_covariate exists, use this column as a cohort effect
    if plex_covariate:
        client.add_cohort_effects_to_design(plex_covariates_list, plex_covariate)
    else:
        client.add_cohort_effects_to_design(cohorts)

    logging.info(f"Samples in {client.cohort_name} data: {len(client.sample_names)}")        
    logging.info(f"Protein groups in {client.cohort_name} data:  {len(client.prot_names)}")
    logging.info(f"Design {client.cohort_name} has beed updated with cohort effects")

    na_count_in_variable, samples_per_class = client.apply_filters(
            min_f=max_na_rate, 
            remove_single_peptide_prots=remove_single_pep_protein
        )
    # add each value from a dict to a list
    total_samples[c] = sum(samples_per_class.values())
    # add both as a tuple to the list
    list_of_na_counts_tuples.append((
        na_count_in_variable.to_dict(orient='index'), samples_per_class))
    logging.info("")
    
logging.info("CLIENT: Filters have been applied...")

24-Mai-24 18:41:00 - root - INFO - Client Center1:	Some protein groups are not in the client list: 131 and will be added.
24-Mai-24 18:41:00 - root - INFO - Client Center1:	Protein groups are validated.
24-Mai-24 18:41:00 - root - INFO - Client Center1:	3 columns are excluded from the design matrix: {'CommonReference', 'Pool', 'Center'}
24-Mai-24 18:41:00 - root - INFO - Client Center1:	Validated 20 samples and 621 proteins.
24-Mai-24 18:41:00 - root - INFO - Client Center1:	Adding cohort effects to the design matrix.
24-Mai-24 18:41:00 - root - INFO - Client Center1:	Adding TMT-plex as a covariate.
24-Mai-24 18:41:00 - root - INFO - Client Center1:	Collinearity will be checked.
24-Mai-24 18:41:00 - root - INFO - Client Center1:	Cohort effects are added to the design matrix.
24-Mai-24 18:41:00 - root - INFO - Samples in Center1 data: 20
24-Mai-24 18:41:00 - root - INFO - Protein groups in Center1 data:  621
24-Mai-24 18:41:00 - root - INFO - Design Center1 has beed updated with cohort 

In [8]:
# SERVER SIDE
keep_proteins = utils.filter_features_na_rate(list_of_na_counts_tuples, max_na_rate)
logging.info(f"SERVER: Proteins after filtering: {len(keep_proteins)}")

24-Mai-24 18:41:00 - root - INFO - SERVER: Proteins after filtering: 532


In [9]:
for c in cohorts:
    client = store_clients[c]
    client.update_prot_names(keep_proteins)

24-Mai-24 18:41:00 - root - INFO - Samples in Center1 data: 20, protein groups: 532
24-Mai-24 18:41:00 - root - INFO - Samples in Center2 data: 19, protein groups: 532
24-Mai-24 18:41:00 - root - INFO - Samples in Center3 data: 20, protein groups: 532


# Normalization

Median normalization

In [10]:
# # if TMT and use_median, compute medians
# if experiment_type == "TMT" and use_median:
    
#     # server nows about total_samples 
#     avg_medians = []

#     for c in cohorts:
#         client = store_clients[c]
#         avg_medians.append(client.compute_medians())
        

In [11]:
# # if TMT and use_median, compute medians
# if experiment_type == "TMT" and use_median:
#     global_median_mean = utils.aggregate_medians(avg_medians, total_samples)

In [12]:
# # if TMT and use_median, compute medians
# if experiment_type == "TMT" and use_median:
#     for c in cohorts:
#         client = store_clients[c]
#         client.mean_median_centering(global_median_mean)

#         # proceed with IRS normalization
#         if experiment_type == "TMT" and use_irs:
#             client.irsNorm_in_silico_single_center()

# LmFit

In [13]:
for c in cohorts:
    client = store_clients[c]
    client.prepare_for_limma(keep_proteins)

24-Mai-24 18:41:00 - root - INFO - Client Center1:	Plex column is removed from the design matrix.
24-Mai-24 18:41:00 - root - INFO - Client Center1:	Log2(x+1) transformed intensities.
24-Mai-24 18:41:00 - root - INFO - Client Center1:	Prepared for limma. Samples: 20, Proteins: 532.
24-Mai-24 18:41:00 - root - INFO - Client Center2:	Plex column is removed from the design matrix.
24-Mai-24 18:41:00 - root - INFO - Client Center2:	Log2(x+1) transformed intensities.
24-Mai-24 18:41:00 - root - INFO - Client Center2:	Prepared for limma. Samples: 19, Proteins: 532.
24-Mai-24 18:41:00 - root - INFO - Client Center3:	Plex column is removed from the design matrix.
24-Mai-24 18:41:00 - root - INFO - Client Center3:	Log2(x+1) transformed intensities.
24-Mai-24 18:41:00 - root - INFO - Client Center3:	Prepared for limma. Samples: 20, Proteins: 532.


In [14]:
masks_list = []

for c in cohorts:
    client = store_clients[c]
    masks_list.append(client.get_mask())

In [15]:
variables = client.design.columns.values

# SERVER SIDE
k = len(variables)
n = len(keep_proteins)
logging.info(f"SERVER: Number of proteins: {n}, number of variables: {k}")

# aggregate mask
global_mask = utils.aggregate_masks(masks_list, n, k, used_SMPC=False)

24-Mai-24 18:41:00 - root - INFO - SERVER: Number of proteins: 532, number of variables: 7
24-Mai-24 18:41:00 - root - INFO - SMPC is not used, aggregating masks


In [16]:
list_of_masks = []

for c in cohorts:
    client = store_clients[c]
    updated_masks = client.updated_mask(global_mask)
    list_of_masks.append(updated_masks)

24-Mai-24 18:41:00 - root - INFO - Client Center1:	Collinearity check and update completed.
24-Mai-24 18:41:00 - root - INFO - Client Center2:	Collinearity check and update completed.
24-Mai-24 18:41:00 - root - INFO - Client Center3:	Collinearity check and update completed.


In [17]:
mask_glob = utils.aggregate_masks(list_of_masks, n, k, second_round=True, used_SMPC=False)

24-Mai-24 18:41:00 - root - INFO - SMPC is not used, aggregating masks


In [18]:
# CLIENT SIDE
XtX_XtY_list = []

for c in cohorts:
    client = store_clients[c]
    logging.info("Start computing XtX and XtY")
    XtX, XtY = client.compute_XtX_XtY()
    XtX_XtY_list.append((XtX, XtY))

    logging.info(f"XtX and XtY have been computed for {client.cohort_name}")
    logging.info(f'Design colnames: {client.design.columns.values}')

24-Mai-24 18:41:00 - root - INFO - Start computing XtX and XtY
24-Mai-24 18:41:00 - root - INFO - XtX and XtY have been computed for Center1
24-Mai-24 18:41:00 - root - INFO - Design colnames: ['heathy' 'FSGS' 'Pool2' 'Pool3' 'Pool5' 'Pool4' 'Pool6']
24-Mai-24 18:41:00 - root - INFO - Start computing XtX and XtY
24-Mai-24 18:41:00 - root - INFO - XtX and XtY have been computed for Center2
24-Mai-24 18:41:00 - root - INFO - Design colnames: ['heathy' 'FSGS' 'Pool2' 'Pool3' 'Pool5' 'Pool4' 'Pool6']
24-Mai-24 18:41:00 - root - INFO - Start computing XtX and XtY
24-Mai-24 18:41:00 - root - INFO - XtX and XtY have been computed for Center3
24-Mai-24 18:41:00 - root - INFO - Design colnames: ['heathy' 'FSGS' 'Pool2' 'Pool3' 'Pool5' 'Pool4' 'Pool6']


In [47]:
# SERVER SIDE
XtX_glob, XtY_glob = utils.aggregate_XtX_XtY(XtX_XtY_list, n, k, used_SMPC=False)
logging.info("SERVER: XtX and XtY have been aggregated")

24-Mai-24 18:40:06 - root - INFO - SMPC is not used, aggregating XtX and XtY
24-Mai-24 18:40:06 - root - INFO - SERVER: XtX and XtY have been aggregated


In [48]:
logging.info('SERVER: Computing beta and stdev')
beta, stdev_unscaled = utils.compute_beta_and_stdev(XtX_glob, XtY_glob, n, k, mask_glob)

24-Mai-24 18:40:06 - root - INFO - SERVER: Computing beta and stdev
24-Mai-24 18:40:06 - root - INFO - Detected partial NA coefficients for 208 probe(s).


In [49]:
# CLIENT SIDE
list_of_sse_cov_coef = []

for c in cohorts:
    client = store_clients[c]
    logging.info(f"Start computation of SSE and cov_coef...")
    SSE, cov_coef = client.compute_SSE_and_cov_coef(beta, mask_glob)
    intensities_sum = np.array(client.sum_intensities())
    n_measurements = np.array(client.get_not_na())
    list_of_sse_cov_coef.append((SSE, cov_coef, intensities_sum, n_measurements))
    logging.info(f"SSE and cov_coef have been computed for {client.cohort_name}")

24-Mai-24 18:40:06 - root - INFO - Start computation of SSE and cov_coef...
24-Mai-24 18:40:06 - root - INFO - SSE and cov_coef have been computed for Center1
24-Mai-24 18:40:06 - root - INFO - Start computation of SSE and cov_coef...
24-Mai-24 18:40:06 - root - INFO - SSE and cov_coef have been computed for Center2
24-Mai-24 18:40:06 - root - INFO - Start computation of SSE and cov_coef...
24-Mai-24 18:40:06 - root - INFO - SSE and cov_coef have been computed for Center3


In [50]:
# SERVER SIDE
logging.info("SERVER: Aggregating SSE and cov_coef...")
SSE, cov_coef, Amean, n_measurements = utils.aggregate_SSE_and_cov_coef(
        list_of_sse_cov_coef, n, k, False, len(store_clients))


24-Mai-24 18:40:06 - root - INFO - SERVER: Aggregating SSE and cov_coef...


In [51]:
# SERVER SIDE
logging.info("SERVER: Aggregating SSE and cov_coef...")
SSE, cov_coef, Amean, n_measurements = utils.aggregate_SSE_and_cov_coef(
        list_of_sse_cov_coef, n, k, False, len(store_clients))

logging.info('Aggregation of SSE is done, start computing global parameters...')
sigma, cov_coef, df_residual, Amean, var = utils.compute_SSE_and_cov_coef_global(
            cov_coef, SSE, Amean, n_measurements, n, mask_glob
        )


24-Mai-24 18:40:06 - root - INFO - SERVER: Aggregating SSE and cov_coef...
24-Mai-24 18:40:06 - root - INFO - Aggregation of SSE is done, start computing global parameters...


In [52]:
logging.info('Making contrasts...')
contrasts = utils.make_contrasts(target_classes, variables)

logging.info('Fit contrasts...')
contrast_matrix = contrasts.values
ncoef = cov_coef.shape[1]

beta, stdev_unscaled, cov_coef = utils.fit_contrasts(
    beta, 
    contrast_matrix,
    cov_coef, 
    ncoef, 
    stdev_unscaled
)

24-Mai-24 18:40:06 - root - INFO - Making contrasts...
24-Mai-24 18:40:06 - root - INFO - Fit contrasts...
24-Mai-24 18:40:06 - root - INFO - .... orthogonality of design matrix is False


# eBays

In [53]:
logging.info('Start eBays stage...')
results, df_total = utils.moderatedT(
    var, 
    df_residual,
    beta, 
    stdev_unscaled
)
results["AveExpr"] = Amean
logging.info('"Result table is pre-computed...')


24-Mai-24 18:40:06 - root - INFO - Start eBays stage...
24-Mai-24 18:40:06 - root - INFO - "Result table is pre-computed...


In [54]:
logging.info("Computing B statistic...")
results = utils.Bstat(
    df_total, 
    stdev_unscaled, 
    results,
    stdev_coef_lim=np.array([0.1, 4]), proportion=0.01)
logging.info("B statistic has been computed...")


24-Mai-24 18:40:06 - root - INFO - Computing B statistic...
24-Mai-24 18:40:06 - root - INFO - Calculating tail p-values
24-Mai-24 18:40:06 - root - INFO - B statistic has been computed...


In [55]:

logging.info("Computing p-values...")
results = utils.topTableT( 
    results,
    keep_proteins, 
    beta, 
    stdev_unscaled, 
    df_total,
    adjust="fdr_bh", p_value=1.0, lfc=0, confint=0.95)
logging.info("P-values have been computed...")


24-Mai-24 18:40:06 - root - INFO - Computing p-values...
24-Mai-24 18:40:06 - root - INFO - P-values have been computed...


In [56]:
results

,t,AveExpr,B,logFC,CI.L,CI.R,adj.P.Val,P.Value
A0A075B6H9,2.210671,14.685669,-4.521451,0.228686,0.019159,0.438213,0.061827,0.033252
A0A075B6I0,0.681843,13.010482,-6.526756,0.141067,-0.289957,0.572091,0.608181,0.503007
A0A075B6I9;P04211,3.592355,15.193368,-1.049331,0.382801,0.169285,0.596317,0.002470,0.000696
A0A075B6K0;P01717;P01718,-1.451562,13.128078,-5.718139,-0.145901,-0.355983,0.064182,0.236383,0.162603
A0A075B6K2;A0A075B6K5;P80748,-1.199579,14.536304,-6.045198,-0.180040,-0.492721,0.132641,0.335490,0.244050
...,...,...,...,...,...,...,...,...
Q9Y520,-2.685680,12.909182,-3.580968,-0.403983,-0.718379,-0.089587,0.030170,0.014461
Q9Y5C1,-4.562580,15.125914,2.000507,-0.438139,-0.630553,-0.245724,0.000158,0.000028
Q9Y5Y7,-3.533014,17.408097,-1.222250,-0.305987,-0.479525,-0.132449,0.002783,0.000837
Q9Y6R7,-5.208972,19.885664,4.213034,-0.314997,-0.436166,-0.193828,0.000028,0.000003


In [ ]:
# if not using counts - stop here and save the results
if not use_counts:
    logging.info("Writing results to file...")
    logging.info("File: " + f"{output_path}/{result_name}")
    output_file = f"{output_path}/{result_name}"
    # results.to_csv(output_file, sep="\t")
    logging.info(f"Results have been saved to {output_file}")

# fit counts

In [ ]:
# SERVER SIDE
if use_counts:
    global_min_counts = global_min_counts[keep_proteins] + 1
    logging.info(f"Size of global min counts: {global_min_counts.shape}")

    logging.info("Start spectral count eBayes stage...")
    results = utils.spectral_count_ebayes(
        results, 
        global_min_counts, 
        keep_proteins,
        beta, 
        stdev_unscaled,
        df_residual, 
        sigma,
        return_sorted=False, fit_method="loess")
    logging.info("Spectral count eBayes stage is done...")

24-Mai-24 10:56:06 - root - INFO - Size of global min counts: (350,)
24-Mai-24 10:56:06 - root - INFO - Start spectral count eBayes stage...
24-Mai-24 10:56:06 - root - INFO - Fitting LOWESS curve...
24-Mai-24 10:56:06 - root - INFO - min_count: (350,), log_var: (350,)
24-Mai-24 10:56:06 - root - INFO - Spectral count eBayes stage is done...


In [ ]:
# SERVER SIDE
if use_counts:
    logging.info("Writing results...")
    output_file = f"{output_path}/{result_name}"
    results.to_csv(output_file, sep="\t")
    logging.info(f"Results have been saved to {output_file}")

24-Mai-24 10:56:06 - root - INFO - Writing results...
24-Mai-24 10:56:06 - root - INFO - Results have been saved to /home/yuliya/repos/cosybio/FedProt/evaluation/TMT_data//01_smaller_lib_balanced_PG_MajorPG/results/DPE.csv
